In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Qiskit Imports
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

# Sklearn Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from dataset_build.clf_dtSet_hugo import build_purity_classification_dataset

In [18]:
N_SHOTS = 1000

df, X, y = build_purity_classification_dataset(
    n_shots=N_SHOTS,
    n_states_total=500,
    mixed_proportion=0.5  # proportion of mixed states
)

print(f"Dataset shape: {df.shape}")
print(f"Labels distribution:\n{y.value_counts()}")
print(f"\nDataset columns:\n{df.columns.tolist()}")
print(f"\X columns (à revoir):\n{X.columns.tolist()}")

df.head()

Dataset shape: (500, 14)
Labels distribution:
label_purity
1    250
0    250
Name: count, dtype: int64

Dataset columns:
['X_mean', 'Y_mean', 'Z_mean', 'X_real', 'Y_real', 'Z_real', 'theta_ideal', 'phi_ideal', 'X_ideal', 'Y_ideal', 'Z_ideal', 'bloch_radius_real', 'is_pure', 'label_purity']
\X columns (à revoir):
['X_mean', 'Y_mean', 'Z_mean', 'bloch_radius_real']


,X_mean,Y_mean,Z_mean,X_real,Y_real,Z_real,theta_ideal,phi_ideal,X_ideal,Y_ideal,Z_ideal,bloch_radius_real,is_pure,label_purity
0,-0.574,-0.684,0.468,-0.554934,-0.679425,0.480031,1.070106,4.027506,-0.554934,-0.679425,0.480031,1.000000,True,1
1,0.494,0.104,-0.868,0.476936,0.092993,-0.872718,2.634180,0.192563,0.476936,0.092993,-0.874005,0.998875,True,1
2,-0.052,-0.644,0.088,-0.112629,-0.658147,0.063813,1.468646,4.462416,-0.246088,-0.963868,0.101972,0.670757,False,0
3,0.552,-0.254,-0.814,0.542647,-0.277761,-0.792706,2.486032,5.810093,0.542647,-0.277761,-0.792706,1.000000,True,1
4,0.988,0.118,-0.082,0.984119,0.159582,-0.077734,1.648609,0.160758,0.984119,0.159582,-0.077734,1.000000,True,1


In [19]:
features = ['X_mean', 'Y_mean', 'Z_mean']
target = 'label_purity'

X = df[features].values
y = df[target].values

# IMPORTANT: Quantum rotations expect values roughly in [0, 2pi] or [-1, 1].
# Scaling helps the SVM converge and maps data effectively to angles.
scaler = MinMaxScaler(feature_range=(-np.pi, np.pi))
X_scaled = scaler.fit_transform(X)

# Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [20]:
num_qubits = 1
num_features = 3

# Define parameters
x_params = ParameterVector('x', num_features)

# Create a circuit: 1 qubit
feature_map = QuantumCircuit(num_qubits)

# ENCODING STRATEGY: 
# Map classical x, y, z to rotations Rx, Ry, Rz on the single qubit.
# We can add 'reps' (repetitions) to make the model more expressive.
for _ in range(3): # reps=2
    feature_map.h(0)            # Start with superposition
    feature_map.rz(x_params[0], 0)
    feature_map.ry(x_params[1], 0)
    feature_map.rx(x_params[2], 0)

print("--- Quantum Feature Map Circuit ---")
print(feature_map)

--- Quantum Feature Map Circuit ---
   ┌───┐┌──────────┐┌──────────┐┌──────────┐┌───┐┌──────────┐┌──────────┐»
q: ┤ H ├┤ Rz(x[0]) ├┤ Ry(x[1]) ├┤ Rx(x[2]) ├┤ H ├┤ Rz(x[0]) ├┤ Ry(x[1]) ├»
   └───┘└──────────┘└──────────┘└──────────┘└───┘└──────────┘└──────────┘»
«   ┌──────────┐┌───┐┌──────────┐┌──────────┐┌──────────┐
«q: ┤ Rx(x[2]) ├┤ H ├┤ Rz(x[0]) ├┤ Ry(x[1]) ├┤ Rx(x[2]) ├
«   └──────────┘└───┘└──────────┘└──────────┘└──────────┘


In [21]:
# ==========================================
# 3. QUANTUM KERNEL & QSVC
# ==========================================

# Create the Fidelity Quantum Kernel
# This calculates |<psi(x)|psi(y)>|^2 automatically
q_kernel = FidelityQuantumKernel(feature_map=feature_map)

# Create the Quantum Support Vector Classifier
# We pass the kernel directly to the SVC
qsvc = QSVC(quantum_kernel=q_kernel)

print("\nTraining QSVC... (This involves running quantum circuits)")
qsvc.fit(X_train, y_train)


Training QSVC... (This involves running quantum circuits)


,quantum_kernel,<qiskit_machi...0029DC620B6D0>
,C,1.0
,break_ties,False
,cache_size,200
,class_weight,None
,coef0,0.0
,decision_function_shape,'ovr'
,degree,3
,gamma,'scale'
,max_iter,-1
,probability,False


In [22]:
print("Predicting on test set...")
y_pred = qsvc.predict(X_test)

print("\n--- Results ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Predicting on test set...

--- Results ---
Accuracy: 0.56

Confusion Matrix:
[[32 18]
 [26 24]]

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.64      0.59        50
           1       0.57      0.48      0.52        50

    accuracy                           0.56       100
   macro avg       0.56      0.56      0.56       100
weighted avg       0.56      0.56      0.56       100

